In [1]:
import time
from dask.distributed import Client

# Create a Dask client with 8 workers and 2 threads per worker
client = Client(n_workers=8, threads_per_worker=1, memory_limit='2GB')

# Print out the details of the Dask cluster
print(client)

<Client: 'tcp://127.0.0.1:36967' processes=8 threads=8, memory=14.90 GiB>


In [2]:
from google.cloud import storage
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator
from scipy.stats import chi2
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, ExtraTreeRegressor, ExtraTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesRegressor, ExtraTreesClassifier, AdaBoostRegressor
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

from scipy.stats.mstats import winsorize
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings("ignore") # Suppress all warnings
pd.options.display.float_format = '{:.2f}'.format

In [3]:
# Initialize BigQuery and Storage clients
storage_client = storage.Client()

print("Authentication successful!")
storage_client = storage.Client()

Authentication successful!


In [4]:
import dask.dataframe as dd

# Load CSV from GCS with explicit data types
df = dd.read_csv(
    "gs://samplebucketvrs/Intelligence Analytics/og4/2019.csv",
    blocksize="100MB",  
    assume_missing=True,  # Ensures int columns with missing values convert to float
    low_memory=False  # Reduces memory overhead
)

In [5]:
# ✅ (Optional) Convert to Parquet for better performance
df.to_parquet("gs://samplebucketvrs/Intelligence Analytics/og4/2019.parquet", write_index=False)
df = dd.read_parquet("gs://samplebucketvrs/Intelligence Analytics/og4/2019.parquet")  # Much faster!

In [6]:
df.head(2)

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,...,ARR_TIME,ARR_DELAY,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 20
0,2019-01-01,9E,3280.00,GNV,ATL,601.00,1.00,22.00,623.00,714.00,...,722.00,-1.00,51.00,300.00,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,9E,3281.00,MSP,CVG,1359.00,-5.00,15.00,1414.00,1629.00,...,1633.00,-36.00,75.00,596.00,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#df = df.drop(columns=["Unnamed: 20"])

In [8]:
# Identify columns with all NaN values
columns_to_drop = [col for col in df.columns if df[col].isna().all().compute()]

# Drop the columns
df = df.drop(columns=columns_to_drop)

In [9]:
df["FL_DATE"] = df["FL_DATE"].astype("datetime64[ns]")

In [10]:
# Identify numerical and categorical columns
numerical_cols = df.select_dtypes(include=["number"]).columns
# ✅ Impute missing values for numerical columns using median
for col in numerical_cols:
    median_value = df[col].median_approximate().compute()  # Use approximate median for efficiency
    df[col] = df[col].fillna(median_value)
    print(f"Imputed median for {col}: {median_value}")

Imputed median for OP_CARRIER_FL_NUM: 2231.0
Imputed median for DEP_TIME: 1329.0
Imputed median for DEP_DELAY: -2.0
Imputed median for TAXI_OUT: 15.0
Imputed median for WHEELS_OFF: 1342.0
Imputed median for WHEELS_ON: 1503.0
Imputed median for TAXI_IN: 6.0
Imputed median for ARR_TIME: 1507.0
Imputed median for ARR_DELAY: -6.0
Imputed median for AIR_TIME: 94.0
Imputed median for DISTANCE: 641.0
Imputed median for CARRIER_DELAY: 0.0
Imputed median for WEATHER_DELAY: 0.0
Imputed median for NAS_DELAY: 3.0
Imputed median for SECURITY_DELAY: 0.0
Imputed median for LATE_AIRCRAFT_DELAY: 3.0


In [11]:
from scipy.stats import normaltest
import dask.dataframe as dd
import numpy as np
from scipy.stats.mstats import winsorize

In [12]:
def detect_outliers_dask(df, method=None, threshold=1.5):
    """
    Detects outliers using IQR or Z-score, with an option to auto-select the best method.
    """
    outlier_percentages = {}
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    if method is None:
        # Take a small sample (10%) efficiently using random_split
        sample_frac = 0.1
        df_sample, _ = df.random_split([sample_frac, 1 - sample_frac])
        df_sample = df_sample.compute()  # Convert sample to Pandas DataFrame

        normality_pvals = df_sample[numeric_cols].apply(lambda x: normaltest(x.dropna())[1])

        if (normality_pvals > 0.05).all():  
            method = "zscore"  # If p > 0.05, assume normal distribution
        else:
            method = "iqr"  # Otherwise, assume non-normal distribution

        print(f"Auto-selected method: {method}")

    # Now, use IQR or Z-score as before
    if method == "iqr":
        for col in numeric_cols:
            quantiles = df[col].quantile([0.25, 0.75]).compute()
            Q1, Q3 = quantiles.loc[0.25], quantiles.loc[0.75]
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR

            mask = (df[col] < lower_bound) | (df[col] > upper_bound)
            outlier_count = mask.sum().compute()
            total_rows = len(df)
            outlier_percentages[col] = (outlier_count / total_rows) * 100

    elif method == "zscore":
        for col in numeric_cols:
            mean, std = df[col].mean().compute(), df[col].std().compute()
            mask = abs((df[col] - mean) / std) > threshold
            outlier_count = mask.sum().compute()
            total_rows = len(df)
            outlier_percentages[col] = (outlier_count / total_rows) * 100

    else:
        raise ValueError("Method must be 'iqr' or 'zscore'.")

    return outlier_percentages


In [13]:
def clean_or_winsorize_dask(df, outlier_percentages, threshold=5):
    """
    Cleans or applies Winsorization based on outlier percentage.
    
    Parameters:
    - df: Dask DataFrame
    - outlier_percentages: Dictionary of outlier percentages per column.
    - threshold: Percentage threshold to decide between cleaning or Winsorization.
    
    Returns:
    - Dask DataFrame with outliers cleaned or Winsorized.
    """
    numeric_cols = list(outlier_percentages.keys())

    # Compute IQR bounds once for all columns
    stats = df[numeric_cols].quantile([0.25, 0.75]).compute()
    Q1, Q3 = stats.loc[0.25], stats.loc[0.75]
    IQR = Q3 - Q1
    lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

    df_out = df.copy()  # Create a copy to avoid modifying the original

    for col in numeric_cols:
        if outlier_percentages[col] <= threshold:
            print(f"Removing {outlier_percentages[col]:.2f}% outliers from column '{col}'.")
            df_out = df_out.assign(**{col: df[col].where((df[col] >= lower_bound[col]) & (df[col] <= upper_bound[col]))})
        else:
            print(f"Applying Winsorization to column '{col}' with {outlier_percentages[col]:.2f}% outliers.")
            df_out = df_out.assign(**{col: df[col].map_partitions(lambda x: winsorize(x, limits=(0.05, 0.05)), meta=(col, "float64"))})

    return df_out


In [14]:
outlier_percentages = detect_outliers_dask(df, method=None)

Auto-selected method: iqr


In [15]:
df_cleaned = clean_or_winsorize_dask(df, outlier_percentages, threshold=5)

Removing 0.00% outliers from column 'OP_CARRIER_FL_NUM'.
Removing 0.00% outliers from column 'DEP_TIME'.
Applying Winsorization to column 'DEP_DELAY' with 13.17% outliers.
Applying Winsorization to column 'TAXI_OUT' with 5.65% outliers.
Removing 0.00% outliers from column 'WHEELS_OFF'.
Removing 0.00% outliers from column 'WHEELS_ON'.
Applying Winsorization to column 'TAXI_IN' with 6.28% outliers.
Removing 0.00% outliers from column 'ARR_TIME'.
Applying Winsorization to column 'ARR_DELAY' with 9.16% outliers.
Applying Winsorization to column 'AIR_TIME' with 5.49% outliers.
Applying Winsorization to column 'DISTANCE' with 5.81% outliers.
Applying Winsorization to column 'CARRIER_DELAY' with 8.92% outliers.
Removing 1.05% outliers from column 'WEATHER_DELAY'.
Applying Winsorization to column 'NAS_DELAY' with 18.06% outliers.
Removing 0.06% outliers from column 'SECURITY_DELAY'.
Applying Winsorization to column 'LATE_AIRCRAFT_DELAY' with 18.25% outliers.


In [16]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask_ml.preprocessing import LabelEncoder
from dask import delayed
from sklearn.preprocessing import OneHotEncoder

categorical_cols = df.select_dtypes(include=["object", "category","string"]).columns
# ✅ Encode categorical variables
for col in categorical_cols:
    unique_count = df[col].nunique().compute()  # Get number of unique values
    
    if unique_count <= 10:
        # **One-Hot Encoding** for low-cardinality categorical columns
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        
        # Apply One-Hot Encoding using map_partitions
        df = df.map_partitions(
            lambda d: d.join(
                pd.DataFrame(
                    encoder.fit_transform(d[[col]]),
                    index=d.index,
                    columns=[f"{col}_{i}" for i in range(unique_count)]
                )
            ), 
            meta=df
        )
        
        df = df.drop(columns=[col])  # Drop original column after encoding
        print(f"Applied One-Hot Encoding to {col}")

    else:
        # **Label Encoding** for high-cardinality categorical columns
        encoder = LabelEncoder()
        df[col] = df[col].map_partitions(lambda x: encoder.fit_transform(x), meta=(col, 'int64'))
        print(f"Applied Label Encoding to {col}")

# ✅ Check the transformed DataFrame
print(df.dtypes)

Applied Label Encoding to OP_UNIQUE_CARRIER
Applied Label Encoding to ORIGIN
Applied Label Encoding to DEST
FL_DATE                datetime64[ns]
OP_UNIQUE_CARRIER               int64
OP_CARRIER_FL_NUM             float64
ORIGIN                          int64
DEST                            int64
DEP_TIME                      float64
DEP_DELAY                     float64
TAXI_OUT                      float64
WHEELS_OFF                    float64
WHEELS_ON                     float64
TAXI_IN                       float64
ARR_TIME                      float64
ARR_DELAY                     float64
AIR_TIME                      float64
DISTANCE                      float64
CARRIER_DELAY                 float64
WEATHER_DELAY                 float64
NAS_DELAY                     float64
SECURITY_DELAY                float64
LATE_AIRCRAFT_DELAY           float64
dtype: object


In [17]:
import dask.dataframe as dd
import numpy as np
import dask.array as da
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier
from joblib import parallel_backend  # Enables Dask parallelism in Scikit-learn

# ✅ Sample 10% of the data
sampled_df = df.sample(frac=0.1, random_state=42)
print("✅ Data Sampled")

# ✅ Separate features (X) and target (y)
target_col = "DEP_DELAY"
y = sampled_df[target_col]
X = sampled_df.drop(columns=[target_col, "FL_DATE"])

# ✅ Standardize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Data Scaled")

# ✅ Train/Test Split (remains Dask arrays)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ✅ Compute in **small batches** to avoid memory overload
batch_size = 150000  # Adjust based on available memory

def batch_compute(dask_array, batch_size):
    """Convert Dask array to NumPy in small chunks to avoid memory overflow."""
    np_array = dask_array.to_dask_array(lengths=True)  # Convert to Dask Array
    n_rows = np_array.shape[0]

    results = []
    for i in range(0, n_rows, batch_size):
        results.append(np_array[i : i + batch_size].compute())  # Compute in chunks

    return np.vstack(results) if results[0].ndim > 1 else np.hstack(results)

# ✅ Convert X_train, X_test, y_train, y_test in small chunks
X_train_np = batch_compute(X_train, batch_size)
X_test_np = batch_compute(X_test, batch_size)
y_train_np = batch_compute(y_train, batch_size)
y_test_np = batch_compute(y_test, batch_size)

print("✅ Data Converted to NumPy Successfully!")

# ✅ Determine task type & use Scikit-learn models with Dask backend
if np.unique(y_train_np).size <= 10:
    model = ExtraTreesClassifier(n_jobs=-1, random_state=42, n_estimators=10)
    task_type = "Classification"
else:
    model = ExtraTreesRegressor(n_jobs=-1, random_state=42, n_estimators=10)
    task_type = "Regression"

# ✅ Train the model using Dask's parallel backend
with parallel_backend("dask"):
    model.fit(X_train_np, y_train_np)

print(f"✅ {task_type} Model Training Complete!")


✅ Data Sampled
✅ Data Scaled
✅ Data Converted to NumPy Successfully!
✅ Regression Model Training Complete!


In [ ]:
import dask.dataframe as dd
import numpy as np
import dask.array as da
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier

import ray
import ray.data as rd
from ray.util.joblib import register_ray  # Enables Ray parallelism in Scikit-learn
from joblib import parallel_backend

# ✅ Initialize Ray
ray.init(num_cpus=8, ignore_reinit_error=True)
register_ray()  # Register Ray with Joblib

# ✅ Sample 10% of the data
sampled_df = df.sample(frac=0.1, random_state=42)
print("✅ Data Sampled")

# ✅ Separate features (X) and target (y)
target_col = "DEP_DELAY"
y = sampled_df[target_col]
X = sampled_df.drop(columns=[target_col, "FL_DATE"])

# ✅ Standardize numerical features using Dask
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Data Scaled")

# ✅ Train/Test Split (Dask Arrays)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ✅ Convert Dask Arrays to Ray DataFrames (Efficient Distributed Processing)
def dask_to_ray(dask_array, batch_size=150000):
    """Convert a Dask array to a Ray Dataset in small chunks."""
    np_array = dask_array.to_dask_array(lengths=True)  # Ensure it's a Dask array
    n_rows = np_array.shape[0]

    results = []
    for i in range(0, n_rows, batch_size):
        results.append(np_array[i : i + batch_size].compute())  # Compute in chunks

    return rd.from_numpy(np.vstack(results)) if results[0].ndim > 1 else rd.from_numpy(np.hstack(results))

# ✅ Convert Dask Data to Ray DataFrames
X_train_ray = dask_to_ray(X_train)
X_test_ray = dask_to_ray(X_test)
y_train_ray = dask_to_ray(y_train)
y_test_ray = dask_to_ray(y_test)

print("✅ Data Converted to Ray DataFrames Successfully!")

# ✅ Store Data in Ray's Object Store for Faster Access
X_train_ref = ray.put(X_train_ray)
X_test_ref = ray.put(X_test_ray)
y_train_ref = ray.put(y_train_ray)
y_test_ref = ray.put(y_test_ray)

# ✅ Determine Task Type and Choose Model
if len(np.unique(y_train_ray.to_pandas())) <= 10:
    model = ExtraTreesClassifier(n_jobs=-1, random_state=42, n_estimators=10)
    task_type = "Classification"
else:
    model = ExtraTreesRegressor(n_jobs=-1, random_state=42, n_estimators=10)
    task_type = "Regression"

# ✅ Train Model Using Ray Parallelism
with parallel_backend("ray"):  # Use Ray for distributed training
    model.fit(ray.get(X_train_ref).to_pandas(), ray.get(y_train_ref).to_pandas())

print(f"✅ {task_type} Model Training Complete with Ray!")

# ✅ OPTIONAL: Save the trained model to Ray's Object Store
model_ref = ray.put(model)


2025-02-10 07:42:49,930	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


✅ Data Sampled


In [15]:
import math
import dask.dataframe as dd
import pandas as pd

def select_top_features_dask(model, X, min_features=5):
    """
    Selects top features based on feature importance from a given model.
    
    Parameters:
    - model: A trained model with a feature_importances_ attribute.
    - X: A Dask or pandas DataFrame of input features.
    - min_features: Minimum number of features to always select.

    Returns:
    - List of selected feature names.
    """
    # Check if the model supports feature importances
    if not hasattr(model, "feature_importances_"):
        raise ValueError("Model does not support feature importances.")
    
    # Ensure X is a Dask DataFrame
    if not isinstance(X, (dd.DataFrame, pd.DataFrame)):
        raise ValueError("X must be a pandas or Dask DataFrame.")

    # Convert to Dask if it is a pandas DataFrame for consistency
    if isinstance(X, pd.DataFrame):
        X = dd.from_pandas(X, npartitions=1)

    # Get feature importances
    feature_importances = model.feature_importances_

    # Create a DataFrame with features and their importance scores
    importance_df = pd.DataFrame({
        'Feature': X.columns,
        'Importance': feature_importances
    }).sort_values(by='Importance', ascending=False).reset_index(drop=True)
    
    # Debugging statement
    print(importance_df)  # This will show the importance DataFrame

    total_features = len(importance_df)

    # Dynamically calculate the number of features to select
    additional_features = math.ceil(0.6 * total_features)
    selected_features_count = min(total_features, max(min_features, additional_features + min_features))

    # Select top features based on importance
    selected_features_df = importance_df.head(selected_features_count)
    selected_features = selected_features_df['Feature'].tolist()
    
    return selected_features

# Example usage (replace model and X appropriately):
print(select_top_features_dask(model, X))


                Feature  Importance
0             ARR_DELAY        0.72
1         CARRIER_DELAY        0.15
2   LATE_AIRCRAFT_DELAY        0.08
3             NAS_DELAY        0.02
4              TAXI_OUT        0.01
5               TAXI_IN        0.00
6              DISTANCE        0.00
7         WEATHER_DELAY        0.00
8              AIR_TIME        0.00
9              DEP_TIME        0.00
10               ORIGIN        0.00
11           WHEELS_OFF        0.00
12    OP_CARRIER_FL_NUM        0.00
13    OP_UNIQUE_CARRIER        0.00
14                 DEST        0.00
15            WHEELS_ON        0.00
16             ARR_TIME        0.00
17       SECURITY_DELAY        0.00
['ARR_DELAY', 'CARRIER_DELAY', 'LATE_AIRCRAFT_DELAY', 'NAS_DELAY', 'TAXI_OUT', 'TAXI_IN', 'DISTANCE', 'WEATHER_DELAY', 'AIR_TIME', 'DEP_TIME', 'ORIGIN', 'WHEELS_OFF', 'OP_CARRIER_FL_NUM', 'OP_UNIQUE_CARRIER', 'DEST', 'WHEELS_ON']


In [15]:
import ray
from ray.util.multiprocessing import Pool
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier, LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, ExtraTreeClassifier, ExtraTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesClassifier, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from xgboost import XGBRegressor
import pandas as pd

ray.init(num_cpus=8,ignore_reinit_error=True)

2025-02-10 06:02:30,862	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.16
Ray version:,2.42.0
Dashboard:,http://127.0.0.1:8265


In [ ]:
@ray.remote
def train_and_evaluate_model(model_name, model, X_train, X_test, y_train, y_test):
    """Trains and evaluates a model, selecting the best features if applicable."""
    model.fit(X_train, y_train)
    selected_features = select_top_features(model, X_train)
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]
    
    model.fit(X_train_selected, y_train)
    score = model.score(X_test_selected, y_test)
    return model_name, score, selected_features

def select_and_apply_model(X, y):
    """Selects the best model using Ray for parallel execution."""
    # Normalize features
    scaler = StandardScaler()
    X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    # Identify task type
    task_type = "regression" if pd.api.types.is_numeric_dtype(y) else "classification"
    
    # Define models
    models = {
        "regression": {
            "LinearRegression": LinearRegression(),
            "Lasso": Lasso(alpha=0.1, random_state=42),
            "Ridge": Ridge(alpha=1.0, random_state=42),
            "DecisionTreeRegressor": DecisionTreeRegressor(),
            "ExtraTreeRegressor": ExtraTreeRegressor(),
            "RandomForestRegressor": RandomForestRegressor(n_estimators=100, random_state=42),
            "ExtraTreesRegressor": ExtraTreesRegressor(n_estimators=100, random_state=42),
            "AdaBoostRegressor": AdaBoostRegressor(n_estimators=50, random_state=42),
            "XGBRegressor": XGBRegressor(n_estimators=100, random_state=42)
        },
        "classification": {
            "LogisticRegression": LogisticRegression(max_iter=200, random_state=42),
            "RidgeClassifier": RidgeClassifier(),
            "DecisionTreeClassifier": DecisionTreeClassifier(),
            "ExtraTreeClassifier": ExtraTreeClassifier(),
            "KNeighborsClassifier": KNeighborsClassifier(),
            "RadiusNeighborsClassifier": RadiusNeighborsClassifier(),
            "RandomForestClassifier": RandomForestClassifier(n_estimators=100, random_state=42),
            "ExtraTreesClassifier": ExtraTreesClassifier(n_estimators=100, random_state=42)
        }
    }
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)
    
    # Train models in parallel
    tasks = [train_and_evaluate_model.remote(name, model, X_train, X_test, y_train, y_test)
             for name, model in models[task_type].items()]
    results = ray.get(tasks)
    
    # Select the best model
    best_model_name, best_model_score, best_features = max(results, key=lambda x: x[1])
    best_model = models[task_type][best_model_name]
    
    print(f"Best {task_type} model: {best_model_name} with score: {best_model_score:.4f}")
    print(f"Selected Features: {best_features}")
    return best_model, best_model_name, best_model_score, best_features

# Example usage
y = df["DEP_DELAY"]
X = df.drop(columns=["DEP_DELAY", "FL_DATE"])
best_model, model_name, model_score, top_features = select_and_apply_model(X, y)